# Laboratorio No. 4
- Ricardo Méndez 21289
- Sara Echeverría 21371
- Francisco Castillo 21562
- Melissa Pérez 21385

Enlace al repositorio: [https://github.com/bl33h/bankCustomerSegmentation]

# Task 1

### ¿Qué son los grafos computacionales? y ¿cuál es su importancia para el cálculo de gradientes en aplicaciones como backpropagation?
Los grafos computacionales son una representación visual y matemática donde sus nodos corresponden a una operación y los arcos los valores de entrada/salida. Se usan comúnmente para descibir algoritmos y modelos de aprendizaje automático. En el cálculo de gradientes toman un papel importante porque proporcionan una representación clara y estructurada de las operaciones. Lo cual ayuda a que las conexiones entre nodos representen el flujo de datos entre las operaciones. Además, los grafos computacionales permiten realizar el cálculo de los gradientes de forma óptima, lo que ayuda a entrenar modelos por medio de algoritmos.

[Understanding Gradient Descent and Backpropagation](https://www.shramos.com/2019/02/understanding-gradient-descent-and_3.html)

### Detalle cuales son los componentes y pasos que conforman una red neuronal. Con esto en mente, ¿cómo mejoraría el perceptrón que hizo en el laboratorio pasado?

### ¿Cómo se selecciona el valor K usando el método de la silueta para el algoritmo de K-Means. Explique las fórmulas (ecuaciones) que lo componen así como las asumpciones, si hay.

### Investigue sobre Principal Component Analysis (PCA) y responda respecto a algoritmos como K-Means ¿Cómo podría ayudarle a mejorar la calidad de sus clusters cuando se usa K-Means?

# Análisis exploratorio

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
plt.style.use('ggplot')

In [2]:
df = pd.read_csv('data/bank_transactions.csv')
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB
None


In [4]:
print(df.describe())

       CustAccountBalance  TransactionTime  TransactionAmount (INR)
count        1.046198e+06     1.048567e+06             1.048567e+06
mean         1.154035e+05     1.570875e+05             1.574335e+03
std          8.464854e+05     5.126185e+04             6.574743e+03
min          0.000000e+00     0.000000e+00             0.000000e+00
25%          4.721760e+03     1.240300e+05             1.610000e+02
50%          1.679218e+04     1.642260e+05             4.590300e+02
75%          5.765736e+04     2.000100e+05             1.200000e+03
max          1.150355e+08     2.359590e+05             1.560035e+06


In [5]:
# relevant columns in the dataframe
features = df[['CustGender','CustAccountBalance', 'TransactionDate','TransactionTime', 'TransactionAmount (INR)']].copy()

### Encoding y otras variables útiles

In [6]:
# cathegorical features and change other values to its numerical representation
features['CustGender'] = features['CustGender'].map({'F': 1, 'M': 0}) # 1 for female and 0 for male
features['TransactionDate'] = pd.to_datetime(features['TransactionDate']) # correct datatype conversion

In [7]:
# function to get the seconds for each transaction in seconds
def timeToSeconds (time):
    hours = time // 10000
    minutes = (time % 10000) // 100
    seconds = time % 100
    return hours * 3600 + minutes * 60 + seconds
features['TransactionTimeSeconds'] = features['TransactionTime'].apply(timeToSeconds)
print(features['TransactionTimeSeconds'])

0          52327
1          51538
2          52032
3          52034
4          65516
           ...  
1048562    67704
1048563    67054
1048564    66793
1048565    67626
1048566    65542
Name: TransactionTimeSeconds, Length: 1048567, dtype: int64


In [8]:
print(features.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 6 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   CustGender               1047466 non-null  float64       
 1   CustAccountBalance       1046198 non-null  float64       
 2   TransactionDate          1048567 non-null  datetime64[ns]
 3   TransactionTime          1048567 non-null  int64         
 4   TransactionAmount (INR)  1048567 non-null  float64       
 5   TransactionTimeSeconds   1048567 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 48.0 MB
None


### Balanceo de clases

En este caso la variable de interés 'CustGender' sí está desbalanceada, por lo que es pertinente aplicar una técnica para corregirlo.

In [9]:
features['CustGender'].value_counts()

0.0    765530
1.0    281936
Name: CustGender, dtype: int64

In [10]:
male = features[features['CustGender'] == 0]
female = features[features['CustGender'] == 1]
male = male.sample(n=len(female), random_state=7)
features = pd.concat([male, female])

In [11]:
features['CustGender'].value_counts()

0.0    281936
1.0    281936
Name: CustGender, dtype: int64

### Scaling
Las magnitudes de los datos varían significativamente, por lo que es necesario escalar los valores de 'TransactionTimeSeconds' y 'TransactionAmount (INR)'.

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
# apply the scaler
features['TransactionTimeSeconds'] = scaler.fit_transform(features['TransactionTimeSeconds'].values.reshape(-1, 1))
meanTransactionTime = features['TransactionTimeSeconds'].mean()

In [14]:
# apply the scaler
features['TransactionAmount (INR)'] = scaler.fit_transform(features['TransactionAmount (INR)'].values.reshape(-1, 1))
meanTransactionTime = features['TransactionAmount (INR)'].mean()

### Feature selection
Debido a la naturaleza del objetivo para este dataframe, el cual es segmentar a clientes de una entidad bancaria para poder así tomar decisiones a nivel comercial y
potenciar las operaciones de la empresa. Los features considerados de utilidad son:

In [ ]:
features['CustGender'] # segmentar por género
features['TransactionAmount (INR)'] # segmentar por el monto de la transacción
features['TransactionTimeSeconds'] # segmentar por el tiempo de la transacción en segundos
features['CustAccountBalance'] # segmentar por el valor de la cuenta

En términos generales, las variables del dataframe no se encontraban en óptimas condiciones para la implementación del modelo no supervisado, por lo que se realizaron acciones como enconding, scaling y balanceo de clases. Los features expuestos anteriormente representan temas de interés para una entidad bancaria, ya que con estos es posible la identificación de fenómenos como las transacciones fraudulentas, permitiendo la visualización de alguna correlación entre el género, el monto de las transacciones realizadas por el cliente, el valor de la cuenta, entre otros. Por dicha razón, se seleccionaron los features: 'CustGender', 'TransactionAmount (INR)', 'TransactionTimeSeconds' y 'CustAccountBalance'.

# Task 2.1 K-Means

# Task 2.2 Mixture Models